In [1]:
from scapy.all import *
from collections import deque

## Bienvenidxs al taller de DNS.

A lo largo del taller van a poder probar los diferentes comandos que nos provee la librería Scapy para realizar consultas DNS. El objetivo es que al finalizar el taller tengan el código necesario para poder hacer los experimentos y el informe.

El proceso de resolución de nombres consiste en sucesivas consultas y respuestas por parte de todos los servidores DNS involucrados. Las consultas suelen ser recursivas cuando las PC quieren resolver un nombre y le preguntan al Resolver local y suelen ser iterativas cuando los Resolvers le pasan las consultas a los servidores Autoritativos responsables de cada zona. Por esa razón, en una consulta determinada, puede haber subconsultas recursivas e iterativas. Además de los servidores autoritativos de cada zona, el sistema DNS no podría funcionar si no existieran
servidores por encima de toda la jerarquía de zonas que funcionen como punto de partida para comenzar las
consultas iterativas. Estos servidores se llaman Root Name Servers y tienen direcciones IP asignadas fijas,
que nunca cambian de manera que no haga falta hacer una consulta DNS para resolverlos porque sino no se
podría empezar. Estos servidores y sus direcciones IP están listados en la siguiente tabla:

Nombre del Servidor|Direcciones IP (IPv4, IPv6)|Entidad propietaria
:------------------|:--------------------------|:------------------
a.root-servers.net|198.41.0.4, 2001:503:ba3e::2:30|Verisign, Inc.
b.root-servers.net|199.9.14.201, 2001:500:200::b|University of Southern California
c.root-servers.net|192.33.4.12, 2001:500:2::c|Cogent Communications
d.root-servers.net|199.7.91.13, 2001:500:2d::d|University of Maryland
e.root-servers.net|192.203.230.10, 2001:500:a8::e|NASA (Ames Research Center)
f.root-servers.net|192.5.5.241, 2001:500:2f::f|Internet Systems Consortium, Inc.
g.root-servers.net|192.112.36.4, 2001:500:12::d0d|US Department of Defense (NIC)
h.root-servers.net|198.97.190.53, 2001:500:1::53|US Army (Research Lab)
i.root-servers.net|192.36.148.17, 2001:7fe::53|Netnod
j.root-servers.net|192.58.128.30, 2001:503:c27::2:30|Verisign, Inc.
k.root-servers.net|193.0.14.129, 2001:7fd::1|RIPE NCC
l.root-servers.net|199.7.83.42, 2001:500:9f::42|ICANN
m.root-servers.net|202.12.27.33, 2001:dc3::35|WIDE Project

### Utilizando Scapy consultar por el registro A de www.dc.uba.ar

In [2]:
def dnsTypeARequest(domainName, dnsServer):
    dns = DNS(rd=1,qd=DNSQR(qname=domainName))
    udp = UDP(sport=RandShort(), dport=53)
    ip = IP(dst=dnsServer)

    answer = sr1( ip / udp / dns , verbose=0, timeout=10)

    if answer is not None and answer.haslayer(DNS) and answer[DNS].qd.qtype == 1:
        
        if answer[DNS].ancount == 0:
            nextSteps = set()
            for i in range( answer[DNS].arcount):
                nextSteps.add(answer[DNS].ar[i].rdata)
            
            return "ANSWER BUT NO FINISHED", nextSteps
        
        else:
            return "FINISHED", answer[DNS].an
    else:
        return "NO ANSWER", None

In [3]:
def DFS_XX(visited, to_Visit):
    while len(to_Visit) != 0:
        nextIP = to_visit.pop()
        visited.add(nextIP)
        finished, result = dnsTypeARequest("www.dc.uba.ar",nextIP)
        if (finished == "FINISHED"):
            return result
        elif (finished == "ANSWER BUT NO FINISHED"):
            for ip in result: 
                if ip not in visited:
                    to_Visit.append(ip)
    raise Exception("No obtuvo respuesta")

In [4]:
visited = set()
to_Visit = deque()
finished, result = dnsTypeARequest("www.dc.uba.ar", "198.41.0.4")

if finished == "NO ANSWER":
    raise Exception("La primera iteración no obtuvo respuesta")
elif finished == "FINISHED":
    raise Exception("La primera iteración obtuvo la respuesta")
else:
    for ip in result: 
        to_Visit.append(ip)
        
result_ip = DFS_XX(visited,to_Visit)

Exception: La primera iteración no obtuvo respuesta

## Ejercicio:
Adaptar el código anterior de manera que, a través de sucesivas consultas iterativas se obtenga el registro
MX de un dominio dado. Para esto, tener en cuenta que en cada consulta DNS puede tener 3 tipos de respuestas: 
1. nos devuelven los servidores DNS a los cuales seguir preguntando
2. nos devuelven la respuesta a la consulta que estamos haciendo
3. nos devuelven el registro SOA de la zona indicando que el registro solicitado no forma parte de la base de datos de nombres de la zona.